In [34]:
from xcube.core.store import find_data_store_extensions
from xcube.core.store import get_data_store_params_schema
from xcube.core.store import new_data_store

In [35]:
import xarray as xr
import numpy as np
from glob import glob
from tqdm import tqdm
import os
from datetime import datetime

In [36]:
store = new_data_store("s3", root="deep-esdl-input")
store_output = new_data_store("s3", root="deep-esdl-output")

In [38]:
datasets = [
    'phytoplankton-primary-production-1M-9km-64x256x256.zarr',
    'phytoplankton-carbon-1M-9km-64x256x256.zarr',
    'oceanic-export-production-1M-9km-64x256x256.zarr'
]

das = [store.open_data(dataset) for dataset in datasets]

In [52]:
for i in [1,2]:
    das[i]["lat"] = das[0].lat
    das[i]["lon"] = das[0].lon

In [54]:
ds = xr.merge(das)

/home/conda/deepesdl/5ad4a98794ade3cd8ed705d9844d59b5d0a992e84fef742056d8dba1693f56e3-20230119-164346-175524-44-cube-gen/lib/python3.9/site-packages/dask/array/core.py:4806: PerformanceWarning: Increasing number of chunks by factor of 17
  result = blockwise(
/home/conda/deepesdl/5ad4a98794ade3cd8ed705d9844d59b5d0a992e84fef742056d8dba1693f56e3-20230119-164346-175524-44-cube-gen/lib/python3.9/site-packages/dask/array/core.py:4806: PerformanceWarning: Increasing number of chunks by factor of 17
  result = blockwise(
/home/conda/deepesdl/5ad4a98794ade3cd8ed705d9844d59b5d0a992e84fef742056d8dba1693f56e3-20230119-164346-175524-44-cube-gen/lib/python3.9/site-packages/dask/array/core.py:4806: PerformanceWarning: Increasing number of chunks by factor of 17
  result = blockwise(
/home/conda/deepesdl/5ad4a98794ade3cd8ed705d9844d59b5d0a992e84fef742056d8dba1693f56e3-20230119-164346-175524-44-cube-gen/lib/python3.9/site-packages/dask/array/core.py:4806: PerformanceWarning: Increasing number of chunk

In [57]:
ds = ds.chunk(dict(time = 64))

In [59]:
additional_attrs = {
    "date_modified": str(datetime.now()),
    "geospatial_lat_max": float(ds.lat.max().values),
    "geospatial_lat_min": float(ds.lat.min().values),
    "geospatial_lat_resolution": abs(float(
        ds.lat[1] - ds.lat[0]
    )),
    "geospatial_lon_max": float(ds.lon.max().values),
    "geospatial_lon_min": float(ds.lon.min().values),
    "geospatial_lon_resolution": abs(float(
        ds.lon[1] - ds.lon[0]
    )),
    "time_coverage_start": str(ds.time[0].values),
    "time_coverage_end": str(ds.time[-1].values),
}

In [60]:
ds.attrs = additional_attrs

In [ ]:
store_output.write_data(ds, "ocean-1M-9km-64x256x256-1.0.0.zarr", replace=True)